In [12]:
import numpy as np 
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import dataloader as da

In [13]:
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = 120

        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, kernel_size = (1, 51), stride = (1, 1), padding = (0, 25), bias = False)
        self.batchnorm1 = nn.BatchNorm2d(16, eps = 1e-03, momentum=0.1, affine= True, track_running_stats=True)
        
        # Layer 2
        self.conv2 = nn.Conv2d(16, 32, kernel_size = (2, 1), stride = (1, 1), groups = 16, bias = False)
        self.batchnorm2 = nn.BatchNorm2d(32, eps = 1e-02, momentum = 0.1, affine = True, track_running_stats = True)
        self.pooling2 = nn.AvgPool2d(kernel_size=(1, 4), stride = (1, 4), padding = 0)
        
        # Layer 3
        self.conv3 = nn.Conv2d(32, 32, kernel_size = (1, 15), stride = (1, 1), padding = (0, 7), bias = False)
        self.batchnorm3 = nn.BatchNorm2d(32, eps = 1e-05, momentum = 0.1, affine = True, track_running_stats = True)
        self.pooling3 = nn.AvgPool2d(kernel_size = (1, 8), stride = (1, 8), padding = 0)
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints. 
        self.fc1 = nn.Linear(in_features = 736, out_features = 1, bias = True)
        
    def forward(self, x):

        # Layer 1
        x = self.conv1(x)
        x = self.batchnorm1(x)
       
        # Layer 2
        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = F.relu(x, 1.0)
        x = self.pooling2(x)
        x = F.dropout(x, 0.25)
       
        # Layer 3
        x = self.conv3(x)
        x = self.batchnorm3(x)
        x = F.relu(x, 1.0)
        x = self.pooling3(x)
        x = F.dropout(x, 0.25)

        
        # FC Layer
        x = x.view(-1, 736)
        x = self.fc1(x)
        x = F.sigmoid(x)
        
        return x

In [14]:
# def evaluate(model, X, Y, params = ["acc"]):
#     results = []
#     batch_size = 90
    
#     predicted = []
    
#     for i in range(int(len(X)/batch_size)):
#         s = i*batch_size
#         e = i*batch_size+batch_size
        
#         inputs = Variable(torch.from_numpy(X[s:e]).cuda(0))
#         pred = model(inputs)
        
#         predicted.append(pred.data.cpu().numpy())
        
        
#     inputs = Variable(torch.from_numpy(X).cuda(0))
#     predicted = model(inputs)
    
#     predicted = predicted.data.cpu().numpy()
    
#     for param in params:
#         if param == 'acc':
#             results.append(accuracy_score(Y, np.round(predicted)))
#         if param == "auc":
#             results.append(roc_auc_score(Y, predicted))
#         if param == "recall":
#             results.append(recall_score(Y, np.round(predicted)))
#         if param == "precision":
#             results.append(precision_score(Y, np.round(predicted)))
#         if param == "fmeasure":
#             precision = precision_score(Y, np.round(predicted))
#             recall = recall_score(Y, np.round(predicted))
#             results.append(2*precision*recall/ (precision+recall))
#     return results

In [15]:
X_train, y_train, X_test, y_test = da.read_bci_data()

(1080, 1, 2, 750) (1080,) (1080, 1, 2, 750) (1080,)


In [52]:
batch_size = 90
net = EEGNet().cuda(0)
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters())
train_output = []
test_output = []
train_acc = []
test_acc = []
train_loss = []
epochs = 300

for epoch in range(epochs):  # loop over the dataset multiple times
    
    running_loss = 0.0
    for i in range(int(len(X_train)/batch_size)):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = torch.from_numpy(X_train[s:e]).float()
        labels = torch.FloatTensor(np.array([y_train[s:e]]).T*1.0)
        
        # wrap them in Variable
        inputs, labels = Variable(inputs.cuda(0)), Variable(labels.cuda(0))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        train_output += outputs.T.tolist()[0]

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.data
    
    for i in range(int(len(X_test)/batch_size)):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = torch.from_numpy(X_test[s:e]).float()
        labels = torch.FloatTensor(np.array([y_test[s:e]]).T*1.0)
        
        # wrap them in Variable
        inputs, labels = Variable(inputs.cuda(0)), Variable(labels.cuda(0))

        # forward + backward
        outputs = net(inputs)
        test_output += outputs.T.tolist()[0]


    train_accuracy = accuracy_score(np.array(y_train), np.array(train_output).round())     
    train_output = []
    test_accuracy = accuracy_score(np.array(y_test), np.array(test_output).round())  
    test_output = []
    train_acc.append(train_accuracy)
    test_acc.append(test_accuracy)
    train_loss.append(running_loss.tolist())
    
# Validation accuracy
print("Total Epoch: ", epochs)
print("Max train_accuracy ", '%.2f' % (max(train_acc)*100), "%")
print("Max test_accuracy  ", '%.2f' % (max(test_acc)*100), "%")
print("min train_loss     ", '%.2f' % (min(train_loss)))

Total Epoch:  300
Max train_accuracy  99.44 %
Max test_accuracy   83.52 %
min train_loss      0.37
